# Test search utils


In [ ]:
import yaml
import inspect
import importlib
import os,sys
import importlib

sys.path.append('..')

from model_discovery.evolution import BuildEvolution
import model_discovery.utils as U




strparams=[
    f"evoname=test_evo_000",
    "scales=14M,31M,70M",
    "selection_ratio=0.25",
    "select_method=random",
    "design_budget=0",
]
evo_system = BuildEvolution(
    strparams=';'.join(strparams),
    do_cache=False,
    # cache_type='diskcache',
)

ptree = evo_system.ptree


In [2]:

refs_s2ids={}
REF_TYPES=['Reference','ReferenceCore','ReferenceWithCode','ReferenceCoreWithTree']

for i in ptree.G.nodes:
    node=ptree.G.nodes[i]['data']
    if node.type in REF_TYPES:
        s2id=node.s2id
        if s2id is not None:
            refs_s2ids[i]=s2id
        else:
            print(i)

refs=ptree.filter_by_type(REF_TYPES)

ntk
kangpt
srt
s4pp
feedbackmem


In [3]:
import requests
import time
from tqdm import tqdm

headers={"X-API-KEY": os.environ['S2_API_KEY']}
paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}'
params={
    "fields": "openAccessPdf,externalIds",
}

# # pdf_urls={}
# external_ids={}
# for i in tqdm(refs_s2ids):
#     s2id=refs_s2ids[i]
#     rsp=requests.get(
#         paper_detail.format(paper_id=s2id),
#         headers=headers,
#         params=params
#     )
#     results = rsp.json()
#     if 'openAccessPdf' in results and results['openAccessPdf'] is not None:
#         pdf_urls[i]=results['openAccessPdf']
#     if 'externalIds' in results and results['externalIds'] is not None:
#         external_ids[i]=results['externalIds']
#     time.sleep(0.1)


# arxiv_ids={}
# for i,j in external_ids.items():
#     if 'ArXiv' in j:
#         arxiv_ids[i]=j['ArXiv']


In [4]:
def get_node(id):
    return ptree.G.nodes[id]['data']


def get_references(id):
    params={
        "fields": "references",
    }
    s2id=refs_s2ids[id]
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results['references']


all_refs={}
for i in tqdm(refs_s2ids):
    all_refs[i]=get_references(i)

refs_set={}
for i in all_refs:
    for ref in all_refs[i]:
        refs_set[ref['paperId']]=ref['title']


100%|██████████| 292/292 [00:41<00:00,  6.99it/s]


In [5]:
def get_paper_detail(s2id):
    params={
        "fields": "title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,authors,tldr,embedding,tldr,openAccessPdf,externalIds,references",
    }
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results

refs_detail={}
save_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','refs2')
for i in tqdm(refs_set):
    save_path=U.pjoin(save_dir,f'{i}.json')
    if U.pexists(save_path):
        refs_detail[i]=U.load_json(save_path)
        continue
    ret=get_paper_detail(i)
    if 'error' not in ret:
        U.save_json(ret,save_path)
    refs_detail[i]=ret
    time.sleep(0.1)


100%|██████████| 4775/4775 [00:01<00:00, 4379.82it/s]


In [6]:
len(refs_detail)

ref2_arxiv={}
ref2_pdfs={}
for i in refs_detail:
    if 'externalIds' in refs_detail[i]:
        if 'ArXiv' in refs_detail[i]['externalIds']:
            ref2_arxiv[i]=refs_detail[i]['externalIds']['ArXiv']
            continue
    if 'openAccessPdf' in refs_detail[i]:
        url=refs_detail[i]['openAccessPdf']
        if url:
            ref2_pdfs[i]=url
        continue
    print(i)
    

None


In [9]:
len(ref2_arxiv)

3172

In [ ]:
import urllib, urllib.request

save_dir=U.pjoin(ptree.lib_dir,'..','pdfs2')
arxiv_pdf='https://arxiv.org/pdf/{arxiv_id}'
U.mkdir(save_dir)
bar=tqdm(ref2_arxiv)
for i in bar:
    bar.set_description(f'Downloading {i}')
    arxiv_id=ref2_arxiv[i]
    url=arxiv_pdf.format(arxiv_id=arxiv_id)
    save_path=U.pjoin(save_dir,f'{i}.pdf')
    if not os.path.exists(save_path):
        pdf=urllib.request.urlretrieve(url, save_path)


# failed_pdfs={}
# for i in tqdm(ref2_pdfs):
#     url=ref2_pdfs[i]['url']
#     save_path=U.pjoin(save_dir,f'{i}.pdf')
#     if not os.path.exists(save_path):
#         try:
#             pdf=urllib.request.urlretrieve(url, save_path)
#         except:
#             failed_pdfs[i]=url
#             print(f'Failed to download {i}')


In [ ]:
pdf[1]

In [ ]:
for i in refs:
    if i not in arxiv_ids:
        print(i)


In [ ]:
save_dir

In [ ]:
import urllib, urllib.request
url = 'http://export.arxiv.org/api/query?search_query="{query}"&start=0&max_results=1'
title = ptree.G.nodes[refs[0]]['data'].title
encoded_title = urllib.parse.quote(title)
url=url.format(query=encoded_title)
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))


In [ ]:
len(pdf_urls)

In [ ]:
import agents.search_utils 
importlib.reload(agents.search_utils)

from agents.search_utils import SuperScholarSearcher



